In [1]:
import nltk
from nltk.corpus import stopwords
from nltk.probability import FreqDist
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from string import punctuation
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from torchsummary import summary
from tqdm import tqdm

In [2]:
RANDOM_STATE = 29

In [3]:
df_train = pd.read_csv('/content/train.csv', index_col='id')
print(df_train.shape)
df_train.head()

(31962, 2)


,label,tweet
id,,
1,0,@user when a father is dysfunctional and is s...
2,0,@user @user thanks for #lyft credit i can't us...
3,0,bihday your majesty
4,0,#model i love u take with u all the time in ...
5,0,factsguide: society now #motivation


Описание датасета:
The objective of this task is to detect hate speech in tweets.
For the sake of simplicity, we say a tweet contains hate speech
if it has a racist or sexist sentiment associated with it.
So, the task is to classify racist or sexist tweets from other tweets.

Formally, given a training sample of tweets and labels, where label '1'
denotes the tweet is racist/sexist and label '0' denotes the tweet is
not racist/sexist, your objective is to predict the labels on the test dataset.

Таким образом, нам нужно будет искать твиты, которые содержат
расистский или сексистский смысл.

In [4]:
df_test = pd.read_csv('/content/test.csv', index_col='id')
print(df_test.shape)
df_test.head()

(17197, 1)


,tweet
id,
31963,#studiolife #aislife #requires #passion #dedic...
31964,@user #white #supremacists want everyone to s...
31965,safe ways to heal your #acne!! #altwaystohe...
31966,is the hp and the cursed child book up for res...
31967,"3rd #bihday to my amazing, hilarious #nephew..."


Так как тестовые данные не содержат меток, то будем использовать только
трейн для обучения и валидации, чтобы можно было оценить качество модели.
Посмотрим на баланс классов:

In [5]:
df_train['label'].value_counts()

0    29720
1     2242
Name: label, dtype: int64

In [6]:
df_train['label'].value_counts()[0] / df_train['label'].value_counts()[1]

13.256021409455842

Как часто бывает в подобных задачах, мы имеем большой дисбаланс классов.
Забегая вперёд, мной была предпринята попытка устранить его путём классического
оверсэмплинга, где обучающий датасет был увеличен сэмплами из себя же, чтобы уравнять
количество объектов обоих классов. Данный подход привёл к очень быстрому переобучению
ввиду и так малого размера исходного датасета, а упрощать модель было уже некуда.
В итоге, от идеи оверсэмплинга пришлось отказаться.

Сделаем разбивку на трейн и валидацию:

In [7]:
df_train, df_val = train_test_split(df_train, 
                                    test_size=0.2, 
                                    random_state=RANDOM_STATE, 
                                    stratify=df_train['label'])

df_train.shape, df_val.shape

((25569, 2), (6393, 2))

Сделаем подготовку текстов:

In [8]:
lemmatizer = WordNetLemmatizer()
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [9]:
puncts = set(punctuation)
# Не будем очищать текст от апострофов, заменим их потом на пробелы,
# т.к. встроенные в nltk английские стопслова и так потом отфильтруют лишнее
puncts = puncts - {"'"}

In [10]:
def preprocess_text(txt):
    txt = str(txt)
    txt = ''.join(char for char in txt if char not in puncts) # очистка от пунктуации
    txt = txt.replace("'", " ")
    txt = txt.lower().split()
    txt = [word for word in txt if word.isalpha()] # очистка от символов и цифр
    txt = [lemmatizer.lemmatize(word) for word in txt] # лемматизация
    txt = [word for word in txt if word not in stopwords.words('english')] # очистка от стопслов
    return ' '.join(txt)

In [11]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [12]:
tqdm.pandas()

df_train['tweet'] = df_train['tweet'].progress_apply(preprocess_text)
df_val['tweet'] = df_val['tweet'].progress_apply(preprocess_text)

100%|██████████| 6393/6393 [00:07<00:00, 821.17it/s]


In [13]:
df_train.head()

,label,tweet
id,,
14553,0,user amazing wait see going cantwait
2563,0,wait new user trailer gamer
12125,0,thriving iam positive affirmation
6326,0,happy new user book lil upset page faded user ...
3996,0,arrive cold rainy english noh first time back ...


Подготовим общий корпус текста:

In [14]:
train_corpus = ''.join(df_train['tweet'].values)

Сделаем токенизацию:

In [15]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [16]:
tokens = word_tokenize(train_corpus)
tokens[:5]

['user', 'amazing', 'wait', 'see', 'going']

Создадим словарь:

In [17]:
MAX_WORDS = 2000
MAX_LEN = 20

In [18]:
dist = FreqDist(tokens)
tokens_top = [items[0] for items in dist.most_common(MAX_WORDS - 1)]

In [19]:
tokens_top[:10]

['user', 'day', 'love', 'u', 'amp', 'like', 'life', 'happy', 'get', 'wa']

In [20]:
vocabulary = {word: count for count, word in dict(enumerate(tokens_top, 1)).items()}

Переведём твиты в набор индексов, добавим паддинг:

In [21]:
def text_to_sequence(txt, maxlen):
    result = []
    tokens = word_tokenize(txt)
    for word in tokens:
        if word in vocabulary:
            result.append(vocabulary[word])

    padding = [0] * (maxlen-len(result))
    return result[-maxlen:] + padding

In [22]:
X_train = np.array([text_to_sequence(txt, MAX_LEN) for txt in df_train['tweet'].values])
X_val = np.array([text_to_sequence(txt, MAX_LEN) for txt in df_val['tweet'].values])

X_train.shape, X_val.shape

((25569, 20), (6393, 20))

In [23]:
print(f"Оригинальная строка: {df_train['tweet'].iloc[5]}")
print(f"Обработанная строка: {X_train[5]}")

Оригинальная строка: found beautiful one bedroom double stall garage patio amp huge kitchen signed lease wait move
Обработанная строка: [ 172   51   19 1233    5  777 1537 1538   68  694    0    0    0    0
    0    0    0    0    0    0]


Инициализируем свёрточную нейросеть:

In [24]:
class Net(nn.Module):
    def __init__(self, vocab_size=2000, embedding_dim=128, out_channel=64, num_classes=1, threshold=0.5):
        super().__init__()
        self.threshold = threshold
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0) 
        self.conv_1 = nn.Conv1d(embedding_dim, out_channel, kernel_size=3, padding='same') 
        self.bn1 = nn.BatchNorm1d(out_channel)
        self.pool = nn.MaxPool1d(2)
        self.relu = nn.ReLU()
        self.linear_1 = nn.Linear(out_channel, num_classes)
        self.dp1d = nn.Dropout1d(0.5)
        self.dp = nn.Dropout(0.5)
        
        
    def forward(self, x):     # Для понимания обозначим размеры входных данных на каждом слое,
                              # используя гиперпараметры по умолчанию и max_len=20
        x = self.embedding(x) # (1, 20) -> (1, 20, 128)       
        x = x.permute(0, 2, 1) # (1, 20, 128) -> (1, 128, 20)
        x = self.conv_1(x) # (1, 128, 20) -> (1, 64, 20)
        x = self.bn1(x)
        x = self.dp1d(x)
        x = self.relu(x)
        x = self.pool(x) # (1, 64, 20) -> (1, 64, 10)
        
        x = torch.max(x, axis=2).values # (1, 64, 10) -> (1, 64)
        x = self.dp(x)
        x = self.linear_1(x) # (1, 64) -> (1, 1)
        x = torch.sigmoid(x)
        return x
    
    def predict(self, x):
        x = torch.IntTensor(x).to(device)
        x = self.forward(x)
        x = torch.squeeze((x > self.threshold).int())
        return x

In [25]:
class DataWrapper(Dataset):
    def __init__(self, data, target):
        self.data = torch.from_numpy(data)
        self.target = torch.from_numpy(target)
        
    def __getitem__(self, index):
        x = self.data[index]
        y = self.target[index]
            
        return x, y
    
    def __len__(self):
        return len(self.data)

In [26]:
BATCH_SIZE = 512

In [27]:
torch.random.manual_seed(RANDOM_STATE)

train_dataset = DataWrapper(X_train, df_train['label'].values)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

val_dataset = DataWrapper(X_val, df_val['label'].values)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=True)

In [28]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

Напишем код сети. Учитывая дисбаланс классов, метрика accuracy нам
не подходит. Вместо неё будем использовать F1-score.

In [29]:
def train_nn(epochs=5, embedding_dim=128, hidden_size=32, lr=1e-2, threshold=0.5, return_model=False):

    torch.random.manual_seed(RANDOM_STATE)
    torch.backends.cudnn.deterministic = True

    net = Net(vocab_size=MAX_WORDS, embedding_dim=embedding_dim, 
              out_channel=hidden_size, threshold=threshold).to(device)
    optimizer = torch.optim.Adam(net.parameters(), lr=lr)
    criterion = nn.BCELoss()

    for epoch in range(epochs):
        train_losses = np.array([])
        test_losses = np.array([])
        tp, fp, tn, fn = 0, 0, 0, 0

        for i, (inputs, labels) in enumerate(train_loader):
            net.train()
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = net(inputs)

            loss = criterion(outputs, labels.float().view(-1, 1))
            loss.backward()
            optimizer.step()

            train_losses = np.append(train_losses, loss.item())

            net.eval()
            outputs = torch.squeeze((net(inputs) > threshold).int())

            tp += ((labels == 1) & (outputs == 1)).sum().item()
            tn += ((labels == 0) & (outputs == 0)).sum().item()
            fp += ((labels == 0) & (outputs == 1)).sum().item()
            fn += ((labels == 1) & (outputs == 0)).sum().item()

        precision = tp / (tp + fp) if (tp + fp) != 0 else 0
        recall = tp / (tp + fn) if (tp + fn) != 0 else 0

        f1_score = 2 * precision * recall / (precision + recall) if (precision + recall) != 0 else 0

        print(f'Epoch [{epoch + 1}/{epochs}]. ' \
              f'Loss: {train_losses.mean():.3f}. ' \
              f'F1-score: {f1_score:.3f}', end='. ')

        tp, fp, tn, fn = 0, 0, 0, 0

        with torch.no_grad():
            for i, (inputs, labels) in enumerate(val_loader):

                inputs, labels = inputs.to(device), labels.to(device)
                outputs = net(inputs)

                loss = criterion(outputs, labels.float().view(-1, 1))
                test_losses = np.append(test_losses, loss.item())

                tp += ((labels == 1) & (torch.squeeze((outputs > threshold).int()) == 1)).sum()
                tn += ((labels == 0) & (torch.squeeze((outputs > threshold).int()) == 0)).sum()
                fp += ((labels == 0) & (torch.squeeze((outputs > threshold).int()) == 1)).sum()
                fn += ((labels == 1) & (torch.squeeze((outputs > threshold).int()) == 0)).sum()

        precision = tp / (tp + fp) if (tp + fp) != 0 else 0
        recall = tp / (tp + fn) if (tp + fn) != 0 else 0

        f1_score = 2 * precision * recall / (precision + recall) if (precision + recall) != 0 else 0

        print(f'Test loss: {test_losses.mean():.3f}. Test F1-score: {f1_score:.3f}. Precision: {precision:.3f}. Recall: {recall:.3f}')

    print('Training is finished!')
    if return_model:
        return net

Обучим модель на 20 эпохах:

In [30]:
train_nn(epochs=20, embedding_dim=128)

Epoch [1/20]. Loss: 0.293. F1-score: 0.003. Test loss: 0.199. Test F1-score: 0.000. Precision: 0.000. Recall: 0.000
Epoch [2/20]. Loss: 0.200. F1-score: 0.122. Test loss: 0.177. Test F1-score: 0.254. Precision: 0.917. Recall: 0.147
Epoch [3/20]. Loss: 0.170. F1-score: 0.430. Test loss: 0.164. Test F1-score: 0.450. Precision: 0.866. Recall: 0.304
Epoch [4/20]. Loss: 0.147. F1-score: 0.557. Test loss: 0.160. Test F1-score: 0.506. Precision: 0.798. Recall: 0.371
Epoch [5/20]. Loss: 0.133. F1-score: 0.629. Test loss: 0.166. Test F1-score: 0.509. Precision: 0.845. Recall: 0.364
Epoch [6/20]. Loss: 0.125. F1-score: 0.657. Test loss: 0.163. Test F1-score: 0.548. Precision: 0.796. Recall: 0.417
Epoch [7/20]. Loss: 0.118. F1-score: 0.681. Test loss: 0.168. Test F1-score: 0.556. Precision: 0.785. Recall: 0.431
Epoch [8/20]. Loss: 0.110. F1-score: 0.714. Test loss: 0.178. Test F1-score: 0.564. Precision: 0.802. Recall: 0.435
Epoch [9/20]. Loss: 0.106. F1-score: 0.737. Test loss: 0.179. Test F1-sc

Видно, что модель быстро переобучается, несмотря на то, что у нас всего
один слой свёртки и один выходной полносвязный, а также два слоя дропаута.
Это связано с малым размером датасета. Сильное снижение количества каналов
свёртки приводит к тому, что модель просто перестаёт обучаться, 32 канала
в данном случае - более-менее оптимальное количество, найденное эмпирически.
Снижение размерности эмбеддингов снижает переобучение, но не устраняет его,
и, в целом, модель хуже показывает себя на тестовых данных.
По значениям лосса и метрики на тесте считаю, что оптимальное значение эпох -
7-9. Также имеем ввиду, что перед нами стоит задача выявления оскорбительных
твитов, а значит метрика Recall имеет важное значение. Обучим нашу модель
заново на 9 эпохах и снизим порог классификации, чтобы выявлять больше
оскорбительных твитов:

In [31]:
my_net = train_nn(epochs=9, threshold=0.25, return_model=True)

Epoch [1/9]. Loss: 0.293. F1-score: 0.210. Test loss: 0.199. Test F1-score: 0.330. Precision: 0.337. Recall: 0.324
Epoch [2/9]. Loss: 0.200. F1-score: 0.424. Test loss: 0.177. Test F1-score: 0.446. Precision: 0.413. Recall: 0.484
Epoch [3/9]. Loss: 0.170. F1-score: 0.538. Test loss: 0.164. Test F1-score: 0.484. Precision: 0.414. Recall: 0.583
Epoch [4/9]. Loss: 0.147. F1-score: 0.595. Test loss: 0.160. Test F1-score: 0.482. Precision: 0.402. Recall: 0.603
Epoch [5/9]. Loss: 0.133. F1-score: 0.623. Test loss: 0.166. Test F1-score: 0.500. Precision: 0.441. Recall: 0.578
Epoch [6/9]. Loss: 0.125. F1-score: 0.651. Test loss: 0.163. Test F1-score: 0.509. Precision: 0.426. Recall: 0.634
Epoch [7/9]. Loss: 0.118. F1-score: 0.677. Test loss: 0.168. Test F1-score: 0.510. Precision: 0.427. Recall: 0.634
Epoch [8/9]. Loss: 0.110. F1-score: 0.689. Test loss: 0.178. Test F1-score: 0.512. Precision: 0.432. Recall: 0.627
Epoch [9/9]. Loss: 0.106. F1-score: 0.699. Test loss: 0.179. Test F1-score: 0.52

Теперь мы находим почти 64% оскорбительных твитов, правда, точность
оставляет желать лучшего, и будет много "ложных тревог". Если бы у нас
было больше данных, то модель обучилась лучше. Проверим, как работает
предсказание модели на единичном примере:

In [32]:
my_net.predict(X_val[np.newaxis, 0])

tensor(1, device='cuda:0', dtype=torch.int32)